In [1]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd

from calenviroscreen_utils import *
from utils import *
import prep_data

In [2]:
frequency_funds_tracts = pd.read_parquet(f'{GCS_FILE_PATH}pd_frequency_funds_tracts.parquet')

In [3]:
frequency_funds_tracts.head(3)

,calitp_itp_id,shape_id,pct_urban,pct_suburban,pct_rural,pct_max,tract_type,ntd_id,transit_provider,_5307_funds,_5311_funds,operating_expenses_total_2019,day_name,departure_hour,route_id,trips_per_hour,mean_runtime_min
0,4,shp-10-09,0.992617,NaN,NaN,0.992617,urban,90014,AC Transit,43522188.0,0.0,475468237.0,Thursday,0,10,2,28.0
1,4,shp-10-09,0.992617,NaN,NaN,0.992617,urban,90014,AC Transit,43522188.0,0.0,475468237.0,Thursday,1,10,0,NaN
2,4,shp-10-09,0.992617,NaN,NaN,0.992617,urban,90014,AC Transit,43522188.0,0.0,475468237.0,Thursday,2,10,0,NaN
